# Завдання 3

Порівняти ефективність алгоритмів пошуку підрядка: Боєра-Мура, Кнута-Морріса-Пратта та Рабіна-Карпа на основі двох текстових файлів (стаття 1, стаття 2). Використовуючи timeit, треба виміряти час виконання кожного алгоритму для двох видів підрядків: одного, що дійсно існує в тексті, та іншого — вигаданого (вибір підрядків за вашим бажанням). На основі отриманих даних визначити найшвидший алгоритм для кожного тексту окремо та в цілому.

In [49]:
import timeit

## Алгоритм Боєра-Мура

In [50]:
def build_shift_table(pattern):
  """Створити таблицю зсувів для алгоритму Боєра-Мура."""
  table = {}
  length = len(pattern)
  # Для кожного символу в підрядку встановлюємо зсув рівний довжині підрядка
  for index, char in enumerate(pattern[:-1]):
    table[char] = length - index - 1
  # Якщо символу немає в таблиці, зсув буде дорівнювати довжині підрядка
  table.setdefault(pattern[-1], length)
  return table

def boyer_moore_search(text, pattern):
  # Створюємо таблицю зсувів для патерну (підрядка)
  shift_table = build_shift_table(pattern)
  i = 0 # Ініціалізуємо початковий індекс для основного тексту

  # Проходимо по основному тексту, порівнюючи з підрядком
  while i <= len(text) - len(pattern):
    j = len(pattern) - 1 # Починаємо з кінця підрядка

    # Порівнюємо символи від кінця підрядка до його початку
    while j >= 0 and text[i + j] == pattern[j]:
      j -= 1 # Зсуваємось до початку підрядка

    # Якщо весь підрядок збігається, повертаємо його позицію в тексті
    if j < 0:
      return i # Підрядок знайдено

    # Зсуваємо індекс i на основі таблиці зсувів
    # Це дозволяє "перестрибувати" над неспівпадаючими частинами тексту
    i += shift_table.get(text[i + len(pattern) - 1], len(pattern))

  # Якщо підрядок не знайдено, повертаємо -1
  return -1



## Алгоритм Рабіна-Карпа

In [51]:
def polynomial_hash(s, base=256, modulus=101):
    """
    Повертає поліноміальний хеш рядка s.
    """
    n = len(s)
    hash_value = 0
    for i, char in enumerate(s):
        power_of_base = pow(base, n - i - 1) % modulus
        hash_value = (hash_value + ord(char) * power_of_base) % modulus
    return hash_value

def rabin_karp_search(main_string, substring):
    # Довжини основного рядка та підрядка пошуку
    substring_length = len(substring)
    main_string_length = len(main_string)
    
    # Базове число для хешування та модуль
    base = 256 
    modulus = 101  
    
    # Хеш-значення для підрядка пошуку та поточного відрізка в основному рядку
    substring_hash = polynomial_hash(substring, base, modulus)
    current_slice_hash = polynomial_hash(main_string[:substring_length], base, modulus)
    
    # Попереднє значення для перерахунку хешу
    h_multiplier = pow(base, substring_length - 1) % modulus
    
    # Проходимо крізь основний рядок
    for i in range(main_string_length - substring_length + 1):
        if substring_hash == current_slice_hash:
            if main_string[i:i+substring_length] == substring:
                return i

        if i < main_string_length - substring_length:
            current_slice_hash = (current_slice_hash - ord(main_string[i]) * h_multiplier) % modulus
            current_slice_hash = (current_slice_hash * base + ord(main_string[i + substring_length])) % modulus
            if current_slice_hash < 0:
                current_slice_hash += modulus

    return -1


## Алгоритм Кнута-Морріса-Пратта

In [52]:
def compute_lps(pattern):
    lps = [0] * len(pattern)
    length = 0
    i = 1

    while i < len(pattern):
        if pattern[i] == pattern[length]:
            length += 1
            lps[i] = length
            i += 1
        else:
            if length != 0:
                length = lps[length - 1]
            else:
                lps[i] = 0
                i += 1

    return lps

def kmp_search(main_string, pattern):
    M = len(pattern)
    N = len(main_string)

    lps = compute_lps(pattern)

    i = j = 0

    while i < N:
        if pattern[j] == main_string[i]:
            i += 1
            j += 1
        elif j != 0:
            j = lps[j - 1]
        else:
            i += 1

        if j == M:
            return i - j

    return -1  # якщо підрядок не знайдено


## Стаття 1

In [75]:
with open("стаття 1.txt", "r") as f:
    text_1 = f.read()

### Рядок є в тексті

In [101]:
substring_1_in = "припинення"

In [102]:
print("Rabin-Karp search: ", timeit.timeit('rabin_karp_search(text_1, substring_1_in)', globals=globals(), number = 100))

Rabin-Karp search:  0.1699261000030674


In [103]:
print("Boyer-Moore search: ", timeit.timeit('boyer_moore_search(text_1, substring_1_in)', globals=globals(), number = 100))

Boyer-Moore search:  0.021776399997179396


In [104]:
print("KMP search: ", timeit.timeit('kmp_search(text_1, substring_1_in)', globals=globals(), number = 100))

KMP search:  0.0698465000023134


### Рядка немає в тексті

In [105]:
substring_1_out = "раовліщцщ"

In [106]:
print("Rabin-Karp search (not in text): ", timeit.timeit('rabin_karp_search(text_1, substring_1_out)', globals=globals(), number = 100))

Rabin-Karp search (not in text):  0.5539539000019431


In [107]:
print("Boyer-Moore search (not in text): ", timeit.timeit('boyer_moore_search(text_1, substring_1_out)', globals=globals(), number = 100))

Boyer-Moore search (not in text):  0.07119160000002012


In [108]:
print("KMP search (not in text): ", timeit.timeit('kmp_search(text_1, substring_1_out)', globals=globals(), number = 100))

KMP search (not in text):  0.22003350000886712


## Стаття 2

In [109]:
with open("стаття 2.txt", "r") as f:
    text_2 = f.read()

### Рядок є в тексті

In [110]:
substring_2_in = "Савватеев"

In [111]:
print("Rabin-Karp search: ", timeit.timeit('rabin_karp_search(text_2, substring_2_in)', globals=globals(), number = 100))

Rabin-Karp search:  0.7512909999932162


In [112]:
print("Boyer-Moore search: ", timeit.timeit('boyer_moore_search(text_2, substring_2_in)', globals=globals(), number = 100))

Boyer-Moore search:  0.10387690000061411


In [113]:
print("KMP search: ", timeit.timeit('kmp_search(text_2, substring_2_in)', globals=globals(), number = 100))

KMP search:  0.28361060000315774


### Рядка немає в тексті

In [114]:
substring_2_out = "раовліщцщ"

In [115]:
print("Rabin-Karp search (not in text): ", timeit.timeit('rabin_karp_search(text_2, substring_2_out)', globals=globals(), number = 100))

Rabin-Karp search (not in text):  0.8927360999950906


In [116]:
print("Boyer-Moore search (not in text): ", timeit.timeit('boyer_moore_search(text_2, substring_2_out)', globals=globals(), number = 100))

Boyer-Moore search (not in text):  0.10494469999684952


In [117]:
  print("KMP search (not in text): ", timeit.timeit('kmp_search(text_2, substring_2_out)', globals=globals(), number = 100))

KMP search (not in text):  0.3223743000125978


# Висновки

Найшвидшим з усіх пошуків виявився Boyer-Moore алгоритм, він показує стабіну швидкодію незалежно від того чи існує рядок у тексті, чи ні.

Два інших алгоритма показують дещо гірші результати, особливо якщо рядка не існує у тексті.